Importing libraries

In [1]:
import pandas as pd
import time
import json
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

Web scrapping with Selenium 

In [2]:
# Create service
webdriver_service = Service(ChromeDriverManager().install())
# Create driver
driver = webdriver.Chrome(service = webdriver_service)

In [3]:
# Go to the the main page of the site
page_url = "https://harchi.info/recipes"
driver.get(page_url)

In [4]:
# # Click on Accept cookies
#time.sleep(3)
#driver.find_element(By.XPATH, '//button[text()="Yes, I am happy"]').click()

In [5]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(5)

In [6]:
#links to all pages
pages = [page_url + '?page=' + str(i) for i in range(1, int(driver.find_element(By.CSS_SELECTOR, 'div.pagination div.pagination-pages-list span:nth-last-child(1) a').text)+1)]
#pages

Recipe class

In [7]:
class Recipe:
    def __init__(self, name, type_dish, time_eat, ingredients, recipe_steps, recipe_link):
        self.name = name
        self.type_dish = type_dish
        self.time_eat = time_eat
        self.ingredients = ingredients
        self.recipe_steps = recipe_steps
        self.recipe_link = recipe_link

Function for getting data from a page

In [8]:
def getting_data(page):
    try:
        driver.get(page)
        name = driver.find_element(By.CSS_SELECTOR, 'h1').text
    except:
        return 'none'
    try:
        type_dish = driver.find_element(By.XPATH, "//label[contains(text(), 'Тип страви')]/following-sibling::span").text
    except:
        type_dish = 'none'
    try:
        time_eat = driver.find_element(By.XPATH, "//label[contains(text(), 'Час споживання')]/following-sibling::span").text
    except:
        time_eat = 'none'
    try:
        ingredients = [el.text for el in driver.find_elements(By.CSS_SELECTOR, 'div.recipe-view-ingredients div.recipe-view-ingredient-block')]
    except:
        ingredients = 'none'
    try:
        recipe_steps = [el.text for el in driver.find_elements(By.CSS_SELECTOR, 'div.recipe-view-step')]
    except:
        recipe_steps = 'none'
    recipe_link = page
    return Recipe(name, type_dish, time_eat, ingredients, recipe_steps, recipe_link)

In [9]:
#getting links to recipes
result_link_list = []

for p in pages:
    driver.get(p)
    result_link_list += [el.get_attribute("href") for el in driver.find_elements(By.CSS_SELECTOR, 'div.recipe-list-item > a.button-green')]

In [10]:
all_recipes = []
for link in result_link_list:
    all_recipes += [getting_data(link)]

In [11]:
data = pd.DataFrame([rec.__dict__ for rec in all_recipes if rec!='none'])
data.head()

652

Saving data

In [15]:
all_data.to_csv('recipes.csv', index=False)